<a href="https://colab.research.google.com/github/Youruler1/Speech-Processing-Lab-Material/blob/main/4_Baum_Welch_Algorithm_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np


#generating initial probabilities

#transition probabilities
transition = np.array([[0.8,0.1],
                       [0.1,0.8]])
#Emission probabilities
emission = np.array([[0.1,0.2,0.7],
                     [0.7,0.2,0.1]])

#defining states and sequence symbols
states = ['H','C']
states_dic = {'H':0, 'C':1}
sequence_syms = {'1':0,'2':1,'3':2}
sequence = ['1','2','3']

#test sequence
test_sequence = '331122313'
test_sequence = [x for x in test_sequence]
test_sequence_back = '313221133'
test_sequence_back = [x for x in test_sequence_back]
#probabilities of going to end state
end_probs = [0.1, 0.1]
#probabilities of going from start state
start_probs = [0.5, 0.5]

In [ ]:
def forward_probs():
    # node values stored during forward algorithm
    node_values_fwd = np.zeros((len(states), len(test_sequence)))

    for i, sequence_val in enumerate(test_sequence):
        for j in range(len(states)):
            # if first sequence value then do this
            if (i == 0):
                node_values_fwd[j, i] = start_probs[j] * emission[j, sequence_syms[sequence_val]]
            # else perform this
            else:
                values = [node_values_fwd[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                          range(len(states))]
                node_values_fwd[j, i] = sum(values)

    #end state value
    #end_state = np.multiply(node_values_fwd[:, -1], end_probs)
    end_state_val = sum(node_values_fwd[:, -1])
    return node_values_fwd,end_state_val

In [ ]:
#function to find backward probabilities
import numpy as np

def backward_probs1():
    # node values stored during backward algorithm
    node_values_bwd = np.zeros((len(states), len(test_sequence_back)))

    # Initialize last column to 1
    node_values_bwd[:, -1] = 1

    # Iterate backwards over the sequence
    for i in range(len(test_sequence_back) - 2, -1, -1):
        for j in range(len(states)):
            values = [
                node_values_bwd[k, i+1] *
                emission[k, sequence_syms[test_sequence_back[i+1]]] *
                transition[j, k]
                for k in range(len(states))
            ]
            node_values_bwd[j, i] = sum(values)

    return node_values_bwd


In [ ]:
def si_probs(forward, backward, forward_val):

    si_probabilities = np.zeros((len(states), len(test_sequence)-1, len(states)))

    for i in range(len(test_sequence)-1):
        for j in range(len(states)):
            for k in range(len(states)):
                si_probabilities[j,i,k] = ( forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) \
                                                    / forward_val
    return si_probabilities

In [ ]:
#function to find gamma probabilities
def gamma_probs(forward, backward, forward_val):

    gamma_probabilities = np.zeros((len(states), len(test_sequence)))

    for i in range(len(test_sequence)):
        for j in range(len(states)):
            #gamma_probabilities[j,i] = ( forward[j,i] * backward[j,i] * emission[j,sequence_syms[test_sequence[i]]] ) / forward_val
            gamma_probabilities[j, i] = (forward[j, i] * backward[j, i]) / forward_val

    return gamma_probabilities

In [ ]:
#new
for iteration in range(2000):

    print('\nIteration No: ', iteration + 1)
    # print('\nTransition:\n ', transition)
    # print('\nEmission: \n', emission)

    #Calling probability functions to calculate all probabilities
    fwd_probs, fwd_val = forward_probs()
    bwd_probs= backward_probs1()
    si_probabilities = si_probs(fwd_probs, bwd_probs, fwd_val)
    gamma_probabilities = gamma_probs(fwd_probs, bwd_probs, fwd_val)

    # print('Forward Probs:')
    # print(np.matrix(fwd_probs))
    #
    # print('Backward Probs:')
    # print(np.matrix(bwd_probs))
    #
    # print('Si Probs:')
    # print(si_probabilities)
    #
    # print('Gamma Probs:')
    # print(np.matrix(gamma_probabilities))

    #caclculating 'a' and 'b' matrices
    a = np.zeros((len(states), len(states)))
    b = np.zeros((len(states), len(sequence_syms)))

    #'a' matrix
    for j in range(len(states)):
        for i in range(len(states)):
            for t in range(len(test_sequence)-1):
                a[j,i] = a[j,i] + si_probabilities[j,t,i]

            denomenator_a = [si_probabilities[j, t_x, i_x] for t_x in range(len(test_sequence) - 1) for i_x in range(len(states))]
            denomenator_a = sum(denomenator_a)

            if (denomenator_a == 0):
                a[j,i] = 0
            else:
                a[j,i] = a[j,i]/denomenator_a

    #'b' matrix
    for j in range(len(states)): #states
        for i in range(len(sequence)): #seq
            indices = [idx for idx, val in enumerate(test_sequence) if val == sequence[i]]
            numerator_b = sum( gamma_probabilities[j,indices] )
            denomenator_b = sum( gamma_probabilities[j,:] )

            if (denomenator_b == 0):
                b[j,i] = 0
            else:
                b[j, i] = numerator_b / denomenator_b


    print('\nMatrix a:\n')
    print(np.matrix(a.round(decimals=4)))
    print('\nMatrix b:\n')
    print(np.matrix(b.round(decimals=4)))

    transition = a
    emission = b

    new_fwd_temp, new_fwd_temp_val = forward_probs()
    print('New forward probability: ', new_fwd_temp_val)
    diff =  np.abs(fwd_val - new_fwd_temp_val)
    print('Difference in forward probability: ', diff)

    if (diff < 0.0000001):
        break


Iteration No:  1

Matrix a:

[[0.8341 0.1659]
 [0.1213 0.8787]]

Matrix b:

[[0.1178 0.0674 0.8148]
 [0.4887 0.4578 0.0535]]
New forward probability:  2.7998602619038324e-05
Difference in forward probability:  2.1067814047678323e-05

Iteration No:  2

Matrix a:

[[0.6974 0.3026]
 [0.2949 0.7051]]

Matrix b:

[[0.12   0.0157 0.8643]
 [0.4691 0.4913 0.0396]]
New forward probability:  5.849884477176151e-05
Difference in forward probability:  3.0500242152723186e-05

Iteration No:  3

Matrix a:

[[0.641  0.359 ]
 [0.3686 0.6314]]

Matrix b:

[[0.1171 0.0052 0.8777]
 [0.4756 0.4927 0.0318]]
New forward probability:  6.473650688540742e-05
Difference in forward probability:  6.237662113645915e-06

Iteration No:  4

Matrix a:

[[0.61  0.39 ]
 [0.394 0.606]]

Matrix b:

[[0.1144 0.0023 0.8833]
 [0.4858 0.4867 0.0275]]
New forward probability:  6.7170203154887e-05
Difference in forward probability:  2.4336962694795807e-06

Iteration No:  5

Matrix a:

[[0.5867 0.4133]
 [0.3978 0.6022]]

Matrix b